In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
    """Given two numbers a and b this tool returns their product"""
    return a * b

In [3]:
print(multiply.invoke({"a":4, "b":2}))

8


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given two numbers a and b this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [7]:
# tool binding
llm = ChatOpenAI()

In [8]:
# can bind tools with specific llms only
llm_with_tools = llm.bind_tools([multiply]) # can add more tools in the list

In [9]:
llm_with_tools.invoke("Hi how are you?") # no tool required for this query

AIMessage(content="Hello! I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30802794-61b7-44f1-8c50-d1fc89c0ef60-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
# tool calling
llm_with_tools.invoke("Can you multiply 3 with 10?").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_NrTG3jzshTa7pMA0KPBqjB87',
 'type': 'tool_call'}

In [12]:
# output of - llm_with_tools.invoke("Can you multiply 3 with 10?")

# AIMessage(content='', 
# additional_kwargs={'tool_calls': [{'id': 'call_DoPMyPFse2Ji54MSR7c5VN2K', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 
# 'type': 'function'}], 'refusal': None}, 
# response_metadata={'token_usage': {'completion_tokens': 17, 
# 'prompt_tokens': 62, 'total_tokens': 79, 
# 'completion_tokens_details': {'accepted_prediction_tokens': 0, 
# 'audio_tokens': 0, 'reasoning_tokens': 0, 
# 'rejected_prediction_tokens': 0}, 
# 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 
# 'model_name': 'gpt-3.5-turbo-0125', 
# 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, 
# id='run-06c0f936-ea81-4164-a8a8-cb97e862133c-0', 
# tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_DoPMyPFse2Ji54MSR7c5VN2K', 'type': 'tool_call'}], 
# usage_metadata={'input_tokens': 62, 'output_tokens': 17, 
# 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 
# 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
result = llm_with_tools.invoke("Can you multiply 3 with 10?").tool_calls[0]

In [14]:
# tool execution
# calling a tool with arguments returns the result
multiply.invoke(result["args"])

30

In [15]:
# calling a tool using a tool call returns the result in 
# the form of tool message -> can send this tool message to the llm

multiply.invoke(result)

ToolMessage(content='30', name='multiply', tool_call_id='call_9BkwSlulEwk5aNDYJaLuHEB9')

In [27]:
# sending tool message to llm
query = HumanMessage("can you multiply 3 with 10")
messages = [query]
result = llm_with_tools.invoke(messages)
messages.append(result) # appending ai message
tool_result = multiply.invoke(result.tool_calls[0]) 
messages.append(tool_result) # appending tool message

In [28]:
messages # contains conversation history

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_d4w9ZUepnBsXSO4sWc7xKrkR', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-70ad6230-78dd-41bd-a8d8-6c1ea9b88fee-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_d4w9ZUepnBsXSO4sWc7xKrkR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 17, 'total_tokens': 78, 'input_token_details': {'audio':

In [29]:
llm_with_tools.invoke(messages).content

'The product of 3 and 10 is 30.'

Currency Conversion Tool

In [76]:
# tool create
# tool 1 -> to fetch the conversion factor
# tool 2 -> to compute the conversion

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between 
    a given base currency and a target currency.
    """
    url = f"https://v6.exchangerate-api.com/v6/298237ff0dfe7fe5fb243bc4/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """ 
    Given a currency conversion rate, this function calculates the
    target currency value from a given base currency value.
    """
    return base_currency_value * conversion_rate

In [77]:
get_conversion_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1748044801,
 'time_last_update_utc': 'Sat, 24 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1748131201,
 'time_next_update_utc': 'Sun, 25 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4133}

In [78]:
convert.invoke({"base_currency_value": 10, "conversion_rate": 85.41})

854.0999999999999

In [79]:
# tool binding
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [80]:
messages = [HumanMessage("What is the conversion factor between USD and INR and based on that can you convert 10 USD to INR?")]
ai_message = llm_with_tools.invoke(messages)

In [81]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_ycJLOl9qvz9GPzQkjCZEOTCZ',
  'type': 'tool_call'}]

In [82]:
messages.append(ai_message)

In [83]:
import json

for tool_call in ai_message.tool_calls:
    # execute the first tool and get the value of conversion rate
    if tool_call["name"] == "get_conversion_factor":
        tool_message_1 = get_conversion_factor.invoke(tool_call)
        print(tool_message_1)
        # fetch the conversion rate
        conversion_rate = json.loads(tool_message_1.content)["conversion_rate"]
        # append tool_message_1 to message list
        messages.append(tool_message_1)
    # execute the second tool using the conversion rate from tool 1
    if tool_call["name"] == "convert":
        # fetch the current argument
        tool_call["args"]["conversion_rate"] = conversion_rate
        tool_message_2 = convert.invoke(tool_call)
        messages.append(tool_message_2)


content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1748044801, "time_last_update_utc": "Sat, 24 May 2025 00:00:01 +0000", "time_next_update_unix": 1748131201, "time_next_update_utc": "Sun, 25 May 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 85.4133}' name='get_conversion_factor' tool_call_id='call_ycJLOl9qvz9GPzQkjCZEOTCZ'


In [84]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR and based on that can you convert 10 INR to USD?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ycJLOl9qvz9GPzQkjCZEOTCZ', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 127, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ae2fc30-4ab9-4932-ae1f-0e54b5e8f34c-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id':

In [85]:
llm_with_tools.invoke(messages).content

''